## <center>CLASS PROJECT FOR THE SPATIAL THOUGHTS<br> 'PYTHON FOUNDATION FOR SPATIAL ANALYSIS' COURSE</center>

<div style="text-align: justify">I recently moved to the city of Kolkata in West Bengal, India. When I started thinking of some idea for this project, the first thing that came to my mind was that I could use Python to get the locations of some of the most famous Durga Puja pandals of Kolkata, find an optimum route to visit all these places starting right from my home, and finally look for all restaurants located within 1 km of all these places I shall visit. And my Durga Puja plan for this year just got sorted! Thanks to Python's spatial analysis!</div>

***

First we shall import all the Python libraries we need for this project.

In [1]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
import requests
import os

### Getting the coordinates and geometry of the places

We shall geocode the places to visit, add my home location to the list and then get the geometry of all these places.

In [2]:
# Creating a list of Kolkata's Durga Puja locations to visit

places = ['Santosh Mitra Square', 
          'Bagbazar', 
          'College Square', 
          'Hindustan Park', 
          'Deshapriya Park', 
          'Ahiritola', 
          'Dum Dum Park', 
          'Lake Town', 
          'Manicktala', 
          'Ekdalia', 
          'Mudiali', 
          'Bosepukur', 
          'Naktala', 
          'Jodhpur Park']


In [3]:
# Creating a Pandas dataframe of the places
places_df = pd.DataFrame(places, columns = ['places'])
places_df

,places
0,Santosh Mitra Square
1,Bagbazar
2,College Square
3,Hindustan Park
4,Deshapriya Park
5,Ahiritola
6,Dum Dum Park
7,Lake Town
8,Manicktala
9,Ekdalia


In [4]:
# Using the Nominatim Geocoding Service from geopy library to geocode the places

# user_id = '<replace with your unique id>'

locator = Nominatim(user_agent=user_id)

# Function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# Applying the geocode function on our places dataframe
places_df['location'] = (places_df['places'] + ', Kolkata, West Bengal, India').apply(geocode)

places_df

,places,location
0,Santosh Mitra Square,"(Santosh Mitra Square, Chandi Chowk East, Kolk..."
1,Bagbazar,"(Bagbazar, Rabindra Sarani, Tiretta Bazaar, Ba..."
2,College Square,"(College Square Park, Tiretta Bazaar, Bow Baza..."
3,Hindustan Park,"(Hindustan Park, Manoharpukur, Kolkata, West B..."
4,Deshapriya Park,"(Deshapriya Park, Manoharpukur, Kolkata, West ..."
5,Ahiritola,"(Ahiritola Street, Shobha Bazar, Kolkata, West..."
6,Dum Dum Park,"(Dum Dum Children Park, Kolkata, Barrackpur - ..."
7,Lake Town,"(Lake town, Radha Gobinda Kar Road, Lake Town,..."
8,Manicktala,"(Frank Ross Pharmacy, P-14, CIT Rd, Manicktala..."
9,Ekdalia,"(Ekdalia, Rash Behari Avenue Connector, Kushit..."


In [5]:
# Extracting coordinates of the places and creating columns for latitude and longitude values
places_df['latitude'] = places_df['location'].apply(lambda loc: loc.latitude)
places_df['longitude'] = places_df['location'].apply(lambda loc: loc.longitude)

# Removing the 'location' column from the places dataframe
places_df = places_df.drop('location', axis = 1)

places_df

,places,latitude,longitude
0,Santosh Mitra Square,22.565862,88.365558
1,Bagbazar,22.605210,88.365204
2,College Square,22.574653,88.363905
3,Hindustan Park,22.518370,88.361186
4,Deshapriya Park,22.518435,88.353515
5,Ahiritola,22.595334,88.355472
6,Dum Dum Park,22.621099,88.397826
7,Lake Town,22.609751,88.402644
8,Manicktala,22.581753,88.390794
9,Ekdalia,22.521226,88.369451


In [6]:
# Coordinates of my home
home_coordinates = (22.707052, 88.463047)

In [7]:
# Creating a Pandas dataframe of just my home location

my_home = pd.DataFrame({'places':'My Home', 
                        'latitude':home_coordinates[0], 
                        'longitude':home_coordinates[1]}, 
                       columns=['places', 'latitude', 'longitude'], 
                       index=[0])

my_home

,places,latitude,longitude
0,My Home,22.707052,88.463047


In [8]:
# Adding my home location to the places dataframe
places_df_concat = pd.concat([my_home, places_df]).reset_index(drop=True)
places_df_concat

,places,latitude,longitude
0,My Home,22.707052,88.463047
1,Santosh Mitra Square,22.565862,88.365558
2,Bagbazar,22.605210,88.365204
3,College Square,22.574653,88.363905
4,Hindustan Park,22.518370,88.361186
5,Deshapriya Park,22.518435,88.353515
6,Ahiritola,22.595334,88.355472
7,Dum Dum Park,22.621099,88.397826
8,Lake Town,22.609751,88.402644
9,Manicktala,22.581753,88.390794


In [9]:
# Creating a GeoPandas geodataframe of the places

# Getting the geometry of the locations 
geometry = gpd.points_from_xy(places_df_concat.longitude, places_df_concat.latitude)

# Projecting directly to WGS 84 UTM Zone 45N which covers West Bengal
places_gdf = gpd.GeoDataFrame(places_df_concat, crs='EPSG:32645', geometry=geometry)

places_gdf

,places,latitude,longitude,geometry
0,My Home,22.707052,88.463047,POINT (88.463 22.707)
1,Santosh Mitra Square,22.565862,88.365558,POINT (88.366 22.566)
2,Bagbazar,22.605210,88.365204,POINT (88.365 22.605)
3,College Square,22.574653,88.363905,POINT (88.364 22.575)
4,Hindustan Park,22.518370,88.361186,POINT (88.361 22.518)
5,Deshapriya Park,22.518435,88.353515,POINT (88.354 22.518)
6,Ahiritola,22.595334,88.355472,POINT (88.355 22.595)
7,Dum Dum Park,22.621099,88.397826,POINT (88.398 22.621)
8,Lake Town,22.609751,88.402644,POINT (88.403 22.610)
9,Manicktala,22.581753,88.390794,POINT (88.391 22.582)


### Mapping the places

Now, we shall visualize our places on a map using the Folium library.

In [10]:
# Getting the central coordinates to position the basemap
x_avg = places_gdf.centroid.x.mean()
y_avg = places_gdf.centroid.y.mean()

y_avg, x_avg

(22.561836789999997, 88.37639351186236)

In [11]:
# Creating the basemap 
kolkata_map = folium.Map(location=[y_avg, x_avg], tiles='CartoDB positron', zoom_start=11)

In [12]:
# Getting the places geodataframe's first row that contains my home location 
home = places_gdf.iloc[0]

# Adding my home location to the map with a red home-icon marker
folium.Marker(location=[home.latitude, home.longitude], 
              popup=home.places, 
              icon=folium.Icon(color='red', icon='home')).add_to(kolkata_map)

# Adding the rest of the places to the map, each with a blue map-marker
places_gdf.iloc[1:].apply(lambda place:folium.Marker(location=[place['latitude'], place['longitude']], 
                                                     popup=place['places'], 
                                                     icon=folium.Icon(color='blue', icon='map-marker')
                                                    ).add_to(kolkata_map), axis=1)

kolkata_map

### Finding and mapping the optimum route to visit the places

Next, we shall use the Optimization service of the OpenRouteService API to get the optimum route to travel from my home to the Durga Puja locations in the city, and then visualize the route on our Kolkata map.

In [13]:
# It is clearly visible on the map that geographically 'Naktala' is the farthest location from my house. 
# Hence, we shall consider it the endpoint of our optimum route.

# Getting the row of 'Naktala' from the places geodataframe
endpoint = places_gdf.iloc[13]
endpoint

places                             Naktala
latitude                          22.47387
longitude                        88.361968
geometry     POINT (88.3619683 22.4738701)
Name: 13, dtype: object

In [14]:
# And my home location will be the starting point of our optimum route.

home

places                           My Home
latitude                       22.707052
longitude                      88.463047
geometry     POINT (88.463047 22.707052)
Name: 0, dtype: object

In [15]:
# Creating the vehicles and jobs parameters for the Optimization API call


# Vehicle to travel
vehicle = [{"id":1,
            "profile":"driving-car",
            "start":[home.longitude, home.latitude],
            "end":[endpoint.longitude, endpoint.latitude]}]


# Places to visit
jobs = []
# For loop to append the places from index 1 to 12, excluding the start('My Home') and end('Naktala') points
for i in range(1,13):
    place_dict = {}
    place = places_gdf.iloc[i]
    place_dict['id'] = i
    place_dict['location'] = [place.longitude, place.latitude]
    jobs.append(place_dict)
    
# Appending the remaining location which is at the last index position of the places geodataframe
last_row = places_gdf.iloc[-1]
jobs.append({'id':13, 'location':[last_row.longitude, last_row.latitude]})

In [16]:
vehicle

[{'id': 1,
  'profile': 'driving-car',
  'start': [88.463047, 22.707052],
  'end': [88.3619683, 22.4738701]}]

In [17]:
jobs

[{'id': 1, 'location': [88.36555761435561, 22.5658619]},
 {'id': 2, 'location': [88.3652044, 22.6052096]},
 {'id': 3, 'location': [88.36390478535333, 22.57465335]},
 {'id': 4, 'location': [88.3611863, 22.5183697]},
 {'id': 5, 'location': [88.35351524759007, 22.5184354]},
 {'id': 6, 'location': [88.3554724, 22.595334]},
 {'id': 7, 'location': [88.39782553063668, 22.6210988]},
 {'id': 8, 'location': [88.4026439, 22.6097509]},
 {'id': 9, 'location': [88.3907936, 22.5817532]},
 {'id': 10, 'location': [88.3694508, 22.521226]},
 {'id': 11, 'location': [88.3464486, 22.5100811]},
 {'id': 12, 'location': [88.3852102, 22.5192503]},
 {'id': 13, 'location': [88.363674, 22.5056055]}]

In [18]:
# Using the Optimization service of the OpenRouteService API 

# ORS_API_KEY = '<replace with your unique ORS API key>'

body = {'jobs':jobs, 'vehicles':vehicle}

headers = {'Accept':'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8', 
           'Authorization':ORS_API_KEY, 
           'Content-Type':'application/json; charset=utf-8'}

# Using the requests library to make the API call
response = requests.post('https://api.openrouteservice.org/optimization', json=body, headers=headers)


# Getting the response in json format if the API request is successful 
if response.status_code == 200:
    print('Request successful.')
    optimization_data = response.json()
else:
    print('Request failed.')

Request successful.


In [19]:
optimization_data

{'code': 0,
 'summary': {'cost': 5082,
  'unassigned': 0,
  'service': 0,
  'duration': 5082,
  'waiting_time': 0,
  'computing_times': {'loading': 83, 'solving': 1}},
 'unassigned': [],
 'routes': [{'vehicle': 1,
   'cost': 5082,
   'service': 0,
   'duration': 5082,
   'waiting_time': 0,
   'steps': [{'type': 'start',
     'location': [88.463047, 22.707052],
     'arrival': 0,
     'duration': 0},
    {'type': 'job',
     'location': [88.39782553063668, 22.6210988],
     'id': 7,
     'service': 0,
     'waiting_time': 0,
     'job': 7,
     'arrival': 1155,
     'duration': 1155},
    {'type': 'job',
     'location': [88.4026439, 22.6097509],
     'id': 8,
     'service': 0,
     'waiting_time': 0,
     'job': 8,
     'arrival': 1429,
     'duration': 1429},
    {'type': 'job',
     'location': [88.3907936, 22.5817532],
     'id': 9,
     'service': 0,
     'waiting_time': 0,
     'job': 9,
     'arrival': 1834,
     'duration': 1834},
    {'type': 'job',
     'location': [88.365204

In [20]:
# Creating a list of coordinates of the locations on the optimum route

optimum_route_points = []

steps = optimization_data['routes'][0]['steps']

for step in steps:
    optimum_route_points.append((step['location'][1], step['location'][0]))
    
optimum_route_points

[(22.707052, 88.463047),
 (22.6210988, 88.39782553063668),
 (22.6097509, 88.4026439),
 (22.5817532, 88.3907936),
 (22.6052096, 88.3652044),
 (22.595334, 88.3554724),
 (22.57465335, 88.36390478535333),
 (22.5658619, 88.36555761435561),
 (22.521226, 88.3694508),
 (22.5192503, 88.3852102),
 (22.5183697, 88.3611863),
 (22.5184354, 88.35351524759007),
 (22.5100811, 88.3464486),
 (22.5056055, 88.363674),
 (22.4738701, 88.3619683)]

In [21]:
# Adding the optimum route as a polyline geometry feature to the Kolkata map
folium.PolyLine(optimum_route_points, color='red', weight=1.5, opacity=2).add_to(kolkata_map)

kolkata_map

### Finding all restaurants located within 1 km of each place to visit

Now, we shall create buffers of 1 km around each place to visit, merge all the buffers into a single polygon feature to get rid of intersecting buffer polygons, load an OpenStreetMap geopackage file containing a points layer of restaurants in Kolkata, and extract only those restaurants which fall within the merged buffer polygon.

In [22]:
# Creating a GeoPandas geoseries of buffers of 1 km around the places to visit
buffer_1km = places_gdf['geometry'][1:].buffer(0.01)

In [23]:
# Creating a new map to visualize the buffers
buffer_map = folium.Map(location=[y_avg, x_avg], tiles='CartoDB positron', zoom_start=11)

In [24]:
# Converting the buffers geoseries into a geojson object to add to the map

buffer_geojson = buffer_1km.to_json()
buffer_geojson = folium.GeoJson(data=buffer_geojson)
buffer_geojson.add_to(buffer_map)

buffer_map

In [25]:
# Merging the buffers into a single polygon
merged_polygon = buffer_1km.unary_union

In [26]:
# Creating a new map for visualizing the merged polygon
merged_polygon_map = folium.Map(location=[y_avg, x_avg], tiles='CartoDB positron', zoom_start=11)

In [27]:
# Converting the merged polygon feature into a GeoPandas geoseries and then into a geojson object to add to the map

merged_polygon_geoj = gpd.GeoSeries(merged_polygon).to_json()
merged_polygon_geoj = folium.GeoJson(data=merged_polygon_geoj)
merged_polygon_geoj.add_to(merged_polygon_map)

merged_polygon_map

In [28]:
# Loading the OSM geopackage file consisting of a points layer of the restaurants in Kolkata 

data_pkg_path = 'data'
filename = 'kolkata_restaurants.gpkg'
path = os.path.join(data_pkg_path, filename)

# Getting the restaurants vector layer as a GeoPandas geodataframe
restaurants_gdf = gpd.read_file(path, layer='kolkata_restaurants')

restaurants_gdf.head()

,full_id,osm_id,osm_type,amenity,cuisine,name,addr:postcode,addr:street,alt_name,website,...,name:es,alcohol,brand:wikipedia:pa,brand:wikipedia:ur,contact:phone,contact:website,official_name,diet:healthy,opening_hours:covid19,geometry
0,n1783122342,1783122342,node,restaurant,indian,Dhaba,,,,,...,,,,,,,,,,POINT (88.36579 22.52822)
1,n1833168504,1833168504,node,restaurant,indian,Banana Leaf,7000 026,73 & 75 Rash Behari Avenue,Komala Vilas,http://www.bananaleaf.in/restaurant.html,...,,,,,,,,,,POINT (88.34932 22.51700)
2,n1833181230,1833181230,node,restaurant,chinese;indian,Bar-B-Que,,Park Street,,,...,,,,,,,,,,POINT (88.35263 22.55308)
3,n2234881738,2234881738,node,restaurant,,Oh! Calcutta,700020,Elgin Road,,https://www.speciality.co.in/,...,,,,,,,,,,POINT (88.35122 22.53806)
4,n2283710597,2283710597,node,restaurant,,Big Max,700071,Russel Street,,,...,,,,,,,,,,POINT (88.35111 22.55377)


In [29]:
# Adding the restaurants to the merged plygon map

restaurants_gdf.apply(lambda restaurant:folium.Circle(radius=30, 
                                                      location=[restaurant['geometry'].y, restaurant['geometry'].x], 
                                                      popup=restaurant['name'], 
                                                      color='green', 
                                                      fill=True, 
                                                      fill_opacity=0.5
                                                     ).add_to(merged_polygon_map), axis=1)

merged_polygon_map

In [30]:
# Generating a Pandas series of boolean values indicating which restaurants intersect the merged polygon
bool_series = restaurants_gdf['geometry'].intersects(merged_polygon)

# Using the boolean series to filter out the required restaurants from the restaurants geodataframe
restaurants_within_1km = restaurants_gdf[bool_series]

# Finally, we have the retaurants located within 1 km of the places to visit
restaurants_within_1km.head()

,full_id,osm_id,osm_type,amenity,cuisine,name,addr:postcode,addr:street,alt_name,website,...,name:es,alcohol,brand:wikipedia:pa,brand:wikipedia:ur,contact:phone,contact:website,official_name,diet:healthy,opening_hours:covid19,geometry
0,n1783122342,1783122342,node,restaurant,indian,Dhaba,,,,,...,,,,,,,,,,POINT (88.36579 22.52822)
1,n1833168504,1833168504,node,restaurant,indian,Banana Leaf,7000 026,73 & 75 Rash Behari Avenue,Komala Vilas,http://www.bananaleaf.in/restaurant.html,...,,,,,,,,,,POINT (88.34932 22.51700)
8,n2560440346,2560440346,node,restaurant,indian,Aminia,,,,,...,,,,,,,,,,POINT (88.36687 22.51681)
13,n3051834901,3051834901,node,restaurant,indian,Kasturi,,,,,...,,,,,,,,,,POINT (88.36731 22.52723)
15,n3410845680,3410845680,node,restaurant,,Thali,,,,,...,,,,,,,,,,POINT (88.38901 22.51576)


In [31]:
# Adding the filtered restaurants to the merged polygon map with yellow color...
# to compare with the rest of the restaurants in green color

restaurants_within_1km.apply(lambda restaurant:folium.Circle(radius=30, 
                                                             location=[restaurant['geometry'].y, restaurant['geometry'].x], 
                                                             popup=restaurant['name'], 
                                                             color='yellow', 
                                                             fill=True, 
                                                             fill_opacity=0.8).add_to(merged_polygon_map), axis=1)

merged_polygon_map

***
### Putting it all together

Finally! We are done with our spatial analysis. Now, it's time to visulaize the Durga Puja places to visit, the optimum route to travel from my home to all these locations, and the restaurants I can reach within 1 km of these places - all together on our Kolkata map.

In [32]:
# Adding the filtered restaurants to the Kolkata map

restaurants_within_1km.apply(lambda restaurant:folium.Circle(radius=30, 
                                                             location=[restaurant['geometry'].y, restaurant['geometry'].x], 
                                                             popup=restaurant['name'], 
                                                             color='green', 
                                                             fill=True, 
                                                             fill_color='yellow', 
                                                             fill_opacity=0.5).add_to(kolkata_map), axis=1)

# Our final map is ready!
kolkata_map